In [ ]:
import uproot
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import awkward as ak
import selection
from helpers import *
%load_ext autoreload

In [ ]:
branches = [
    "slc.vertex.x",
    "slc.vertex.y",
    "slc.vertex.z",
    "slc.truth.index",
    "slc.tmatch.pur",
    "slc.tmatch.eff",
    "slc.truth.iscc",
    "slc.truth.pdg",
    "slc.truth.E",
    "slc.truth.position.x",
    "slc.truth.position.y",
    "slc.truth.position.z",
    "slc.truth.prim.length",
    "slc.truth.prim.pdg",
    "slc.truth.nprim",
    "slc.self",
    "slc.nu_score",
    "slc.fmatch.score",
    "slc.fmatch.time",
    "slc.fmatch_a.score",
    "slc.is_clear_cosmic",
    "reco.ntrk",
    "reco.nshw",
    
    "slc.reco.trk.len",
    "slc.reco.trk.costh",
    "slc.reco.trk.phi",
    "slc.reco.trk.start.x",
    "slc.reco.trk.start.y",
    "slc.reco.trk.start.z",
    "slc.reco.trk.end.x",
    "slc.reco.trk.end.y",
    "slc.reco.trk.end.z",
    "slc.reco.trk.ID",
    "slc.reco.trk.slcID",
    "slc.reco.trk.parent_is_primary",
    "slc.reco.trk.chi2pid0.chi2_muon",
    "slc.reco.trk.chi2pid0.chi2_proton",
    "slc.reco.trk.chi2pid1.chi2_muon",
    "slc.reco.trk.chi2pid1.chi2_proton",
    "slc.reco.trk.chi2pid2.chi2_muon",
    "slc.reco.trk.chi2pid2.chi2_proton",
    "slc.reco.trk.bestplane",
    "slc.reco.trk.mcsP.fwdP_muon",
    "slc.reco.trk.rangeP.p_muon",
    "slc.reco.trk.rangeP.p_proton",
    "slc.reco.trk.crthit.hit.time",
    "slc.reco.trk.crthit.distance",
    "slc.reco.trk.crttrack.angle",
    "slc.reco.trk.crttrack.time",
    "slc.reco.trk.truth.bestmatch.G4ID",
    "slc.reco.trk.truth.bestmatch.energy",
    "slc.reco.trk.truth.p.pdg",
    "slc.reco.trk.truth.p.planeVisE",
    "slc.reco.trk.truth.p.gen.x",
    "slc.reco.trk.truth.p.gen.y",
    "slc.reco.trk.truth.p.gen.z",
    "slc.reco.trk.truth.p.genp.x",
    "slc.reco.trk.truth.p.genp.y",
    "slc.reco.trk.truth.p.genp.z",
    "slc.reco.trk.truth.p.startT",
    "slc.reco.trk.truth.p.end.x",
    "slc.reco.trk.truth.p.end.y",
    "slc.reco.trk.truth.p.end.z",
    "slc.reco.trk.truth.p.length",
    "slc.reco.trk.truth.p.contained",
    "slc.reco.trk.truth.p.wallout",
    "slc.reco.trk.truth.p.end_process",
    "slc.reco.trk.truth.total_deposited_energy",
    "mc.nnu",
    "mc.nu.E",
    "mc.nu.index",
    "mc.nu.pdg",
    "mc.nu.iscc",
    "mc.nu.position.x",
    "mc.nu.position.y",
    "mc.nu.position.z",
    "mc.nu.nprim",
    "mc.nu.prim.length",
    "mc.nu.prim.contained",
    "mc.nu.prim.pdg",
    "mc.nu.prim.genp.x",
    "mc.nu.prim.genp.y",
    "mc.nu.prim.genp.z",
    "mc.nu.prim.start.x",
    
    
    "true_particles.G4ID",
    "true_particles.pdg",
    "true_particles.genp.x",
    "true_particles.genp.y",
    "true_particles.genp.z",
    "true_particles.interaction_id",
    "true_particles.start.x",
    "true_particles.startT",
    
    "pass_flashtrig",
    "crt_hits.time",
    "crt_hits.pe",
    "ncrt_hits",
    
    "nslc",
    "slc.reco.ntrk",
    "hdr.pot",
    "hdr.evt",
    "hdr.fno",
    "hdr.subrun",
    "hdr.run",
    "hdr.ngenevt",
    "hdr.evt"
]

In [ ]:
treenames = [
    "rec",
    "rec.slc",
    "rec.slc.reco.trk",
    "rec.true_particles",
    "rec.crt_hits",
    "rec.mc.nu",
    "rec.mc.nu.prim",
    "rec.reco.trk",
    "rec.reco.trk.chi2pid",
    "rec.reco.trk.truth.matches",
    "rec.slc.truth.prim"
]

In [ ]:
do_save = False
savedir = "./plots/"

mpl.rcParams['figure.figsize'] = [6, 4]
mpl.rcParams['figure.dpi'] = 200

In [ ]:
fname = "/pnfs/sbn/persistent/users/gputnam/MCP2020A-CAF/sbnd-overlay.flat.root"
rootf = uproot.open(fname)

data = {}
for b in branches:
    keyname = "rec." + b # ".".join(b.split(".")[1:])
    for t in treenames:
        if not keyname.startswith(t):
            continue
        try:
            d = rootf["recTree"][t].array(keyname)
        except KeyError:
            continue
        data[b] = d
        break
    else:
        raise KeyError(keyname)

groupings = ["slc.reco.trk", "crt_hits", "mc.nu.prim", "slc.truth.prim"]
for k in data.keys():
    for g in groupings:
        if k.startswith(g):
            data[k] = group(data[k], data[g.replace(g.split(".")[-1], "n"+g.split(".")[-1])])

to_broadcast = ["pass_flashtrig"]
broadcast_over = "nslc"
for k in to_broadcast:
    data[k] = broadcast(data[k], data[broadcast_over])
    
# save as ovrl 
ovrl = data
del data

In [ ]:
is_nu = ovrl["slc.truth.index"] >= 0
is_cosmic = np.invert(is_nu)
is_numu_cc = is_nu & ovrl["slc.truth.iscc"] & (np.abs(ovrl["slc.truth.pdg"]) == 14)

ovrl["slc.numu_cc"] = is_nu & ovrl["slc.truth.iscc"] & (np.abs(ovrl["slc.truth.pdg"]) == 14)
ovrl["slc.cosmic"] = np.invert(is_nu)

def dist(x1, y1, z1, x2, y2, z2):
    return np.sqrt((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)

is_fid_true = selection.InFV(ovrl["slc.truth.position.x"], ovrl["slc.truth.position.y"], ovrl["slc.truth.position.z"])
is_fid_reco = selection.InFV(ovrl["slc.vertex.x"], ovrl["slc.vertex.y"], ovrl["slc.vertex.z"])

ovrl["slc.reco.trk.contained"] = selection.InFV(ovrl["slc.reco.trk.end.x"], ovrl["slc.reco.trk.end.y"], ovrl["slc.reco.trk.end.z"])
ovrl["slc.reco.trk.atslc"] = dist(ovrl["slc.reco.trk.start.x"], ovrl["slc.reco.trk.start.y"], ovrl["slc.reco.trk.start.z"],
                                 ovrl["slc.vertex.x"], ovrl["slc.vertex.y"], ovrl["slc.vertex.z"]) < 10

ovrl["slc.reco.trk.truth.p.is_stopping"] = (ovrl["slc.reco.trk.truth.p.end_process"] == 1) |\
                                    (ovrl["slc.reco.trk.truth.p.end_process"] == 2) |\
                                    (ovrl["slc.reco.trk.truth.p.end_process"] == 3) |\
                                    (ovrl["slc.reco.trk.truth.p.end_process"] == 41)

ovrl["slc.reco.trk.recop"] = ovrl["slc.reco.trk.rangeP.p_muon"] + 0. # clone
ovrl["slc.reco.trk.recop"][np.invert(ovrl["slc.reco.trk.contained"])] = ovrl["slc.reco.trk.mcsP.fwdP_muon"][np.invert(ovrl["slc.reco.trk.contained"])] 

for chi2 in ["chi2_muon", "chi2_proton"]:
    ovrl["slc.reco.trk.bestplane." + chi2] = ovrl["slc.reco.trk.chi2pid2." + chi2]
    ovrl["slc.reco.trk.bestplane." + chi2][ovrl["slc.reco.trk.bestplane"] == 0] = ovrl["slc.reco.trk.chi2pid0." + chi2][ovrl["slc.reco.trk.bestplane"] == 0]
    ovrl["slc.reco.trk.bestplane." + chi2][ovrl["slc.reco.trk.bestplane"] == 1] = ovrl["slc.reco.trk.chi2pid1." + chi2][ovrl["slc.reco.trk.bestplane"] == 1]

# For the cases where two slices match to the same neutrino, figure out which match is "primary"
# Use whichever slice gets more of the deposited energy
ovrl["slc.truth.match_is_primary"] = ovrl["slc.truth.iscc"] & False # clone
# Check the max match for each neutrino match 
# Group each slice by spill
index_spill = group(ovrl["slc.truth.index"], ovrl["nslc"])
eff_spill = group(ovrl["slc.tmatch.eff"], ovrl["nslc"])
primary_spill = group(ovrl["slc.truth.match_is_primary"], ovrl["nslc"])

for i in range(ovrl["slc.truth.index"].max()+1): # consider up to the maximum nuetrinos
    primary_spill = primary_spill | (eff_spill[index_spill==i].max() == eff_spill) & (eff_spill[index_spill==i].max() > 0.)

ovrl["slc.truth.match_is_primary"] = primary_spill.flatten()
# all cosmic matches are primary
ovrl["slc.truth.match_is_primary"] = ovrl["slc.truth.match_is_primary"] | (ovrl["slc.truth.index"] < 0)

In [ ]:
# Verify that the truth matching worked as expected

# bad 
for i in range(ovrl["slc.truth.index"].max()+1):
    bad = ((index_spill == i) & primary_spill).sum() > 1
    assert(not bad.any())
    
# also bad
for i in range(ovrl["slc.truth.index"].max()+1):
    bad = ((index_spill == i).sum() > 0) & (np.invert(ovrl["slc.truth.match_is_primary"].any()))
    assert(not bad.any())

when = ((primary_spill == False) & (index_spill >= 0)).any()

#print(eff_spill[when][0])
#print(index_spill[when][0])
#print(primary_spill[when][0])

In [ ]:
fname = "/pnfs/sbn/persistent/users/gputnam/MCP2020A-CAF/sbnd-intime.flat.root"
rootf = uproot.open(fname)

data = {}
for b in branches:
    keyname = "rec." + b # ".".join(b.split(".")[1:])
    for t in treenames:
        if not keyname.startswith(t):
            continue
        try:
            d = rootf["recTree"][t].array(keyname)
        except KeyError:
            continue
        data[b] = d
        break
    else:
        raise KeyError(keyname)

groupings = ["slc.reco.trk", "crt_hits", "mc.nu.prim", "slc.truth.prim"]
for k in data.keys():
    for g in groupings:
        if k.startswith(g):
            data[k] = group(data[k], data[g.replace(g.split(".")[-1], "n"+g.split(".")[-1])])

to_broadcast = ["pass_flashtrig"]
broadcast_over = "nslc"
for k in to_broadcast:
    data[k] = broadcast(data[k], data[broadcast_over])
    
# save as intc 
intc = data
del data

In [ ]:
intc["slc.numu_cc"] = intc["slc.truth.iscc"] # always False
intc["slc.cosmic"] = np.invert(intc["slc.truth.iscc"]) # always True
intc["slc.truth.match_is_primary"] = intc["slc.cosmic"] # always True

intc["slc.reco.trk.contained"] = selection.InFV(intc["slc.reco.trk.end.x"], intc["slc.reco.trk.end.y"], intc["slc.reco.trk.end.z"])
intc["slc.reco.trk.atslc"] = dist(intc["slc.reco.trk.start.x"], intc["slc.reco.trk.start.y"], intc["slc.reco.trk.start.z"],
                                 intc["slc.vertex.x"], intc["slc.vertex.y"], intc["slc.vertex.z"]) < 10

intc["slc.reco.trk.truth.p.is_stopping"] = (intc["slc.reco.trk.truth.p.end_process"] == 1) |\
                                    (intc["slc.reco.trk.truth.p.end_process"] == 2) |\
                                    (intc["slc.reco.trk.truth.p.end_process"] == 3) |\
                                    (intc["slc.reco.trk.truth.p.end_process"] == 41)

intc["slc.reco.trk.recop"] = intc["slc.reco.trk.rangeP.p_muon"] + 0. # clone
intc["slc.reco.trk.recop"][np.invert(intc["slc.reco.trk.contained"])] = intc["slc.reco.trk.mcsP.fwdP_muon"][np.invert(intc["slc.reco.trk.contained"])] 

for chi2 in ["chi2_muon", "chi2_proton"]:
    intc["slc.reco.trk.bestplane." + chi2] = intc["slc.reco.trk.chi2pid2." + chi2]
    intc["slc.reco.trk.bestplane." + chi2][intc["slc.reco.trk.bestplane"] == 0] = intc["slc.reco.trk.chi2pid0." + chi2][intc["slc.reco.trk.bestplane"] == 0]
    intc["slc.reco.trk.bestplane." + chi2][intc["slc.reco.trk.bestplane"] == 1] = intc["slc.reco.trk.chi2pid1." + chi2][intc["slc.reco.trk.bestplane"] == 1]

In [ ]:
ovrl_pot = NeutrinoPOT(ovrl)
intc_pot = CosmicPOT(intc, ovrl)
goal_pot = 6.6e20

ovrl_pot_scale = goal_pot / ovrl_pot
intc_pot_scale = goal_pot / intc_pot

print(ovrl_pot_scale)
print(intc_pot_scale)
print(intc["slc.cosmic"].all())

In [ ]:
primary_track = selection.get_primary_tracks(ovrl)
primary_track_ind = ak.JaggedArray.fromcounts((primary_track >= 0)*1, primary_track[primary_track >= 0])
true_primary_track = selection.get_true_primary_track(ovrl)
true_primary_track_ind = ak.JaggedArray.fromcounts((true_primary_track >= 0)*1, true_primary_track[true_primary_track >= 0])

In [ ]:
# Set all of the primary track slice variables
keys = list(ovrl.keys())
for k in keys:
    if k.startswith("slc.reco.trk."):
        slc_key = k.replace(".reco.trk.", ".ptrk.")
        ovrl[slc_key] = np.empty(ovrl["slc.nu_score"].shape)
        ovrl[slc_key][:] = np.NaN
        
        is_bool = ovrl[k][0].dtype == "bool"
        d = ovrl[k] + 0 # clone
        ovrl[slc_key][primary_track >= 0] = d[primary_track_ind[primary_track_ind >= 0]].flatten()
        if is_bool:
            ovrl[slc_key] = ovrl[slc_key] == 1 # restore bool-type arrays
ovrl["slc.has_ptrk"] = primary_track >= 0

In [ ]:
primary_track = selection.get_primary_tracks(intc)
primary_track_ind = ak.JaggedArray.fromcounts((primary_track >= 0)*1, primary_track[primary_track >= 0])
true_primary_track = selection.get_true_primary_track(intc)
true_primary_track_ind = ak.JaggedArray.fromcounts((true_primary_track >= 0)*1, true_primary_track[true_primary_track >= 0])

In [ ]:
# Set all of the primary track slice variables
keys = list(intc.keys())
for k in keys:
    if k.startswith("slc.reco.trk."):
        slc_key = k.replace(".reco.trk.", ".ptrk.")
        intc[slc_key] = np.empty(intc["slc.nu_score"].shape)
        intc[slc_key][:] = np.NaN
        
        is_bool = intc[k][0].dtype == "bool"
        d = intc[k] + 0 # clone
        intc[slc_key][primary_track >= 0] = d[primary_track_ind[primary_track_ind >= 0]].flatten()
        if is_bool:
            intc[slc_key] = intc[slc_key] == 1 # restore bool-type arrays
intc["slc.has_ptrk"] = primary_track >= 0

In [ ]:
# Define the Cuts!!!!!
fid_ovrl = selection.fid(ovrl)
nu_score_ovrl = selection.nu_score(ovrl)
f_time_ovrl = selection.f_time(ovrl)
f_score_ovrl = selection.f_score(ovrl)
ptrk_ovrl = selection.ptrk(ovrl)
crttrack_ovrl = selection.crttrack(ovrl)
crthit_ovrl = selection.crthit(ovrl)
crtveto_ovrl = selection.crtveto(ovrl)

In [ ]:
# Define the Cuts!!!!!
fid_intc = selection.fid(intc)
nu_score_intc = selection.nu_score(intc)
f_time_intc = selection.f_time(intc)
f_score_intc = selection.f_score(intc)
ptrk_intc = selection.ptrk(intc)
crttrack_intc = selection.crttrack(intc)
crthit_intc = selection.crthit(intc)
crtveto_intc = selection.crtveto(intc)

In [ ]:
# Check normalization 
neutrino_per_spill = (NEvt(ovrl) * POT_PER_SPILL) / NeutrinoPOT(ovrl)
cosmic_per_spill = (NEvt(intc) * POT_PER_SPILL) / CosmicPOT(intc, ovrl)
print("Spill per Neutrino: %.1f" % (1. / neutrino_per_spill))
print("Spill per Cosmic: %.1f" % (1. / cosmic_per_spill))
print("Neutrino Trig. / Cosmic Trig.: %.1f" % (neutrino_per_spill / cosmic_per_spill) )

In [ ]:
# Show all the neutrinos at truth level

var = ovrl["mc.nu.E"]
fid = selection.InFV(ovrl["mc.nu.position.x"], ovrl["mc.nu.position.y"], ovrl["mc.nu.position.z"])
numu_cc = ovrl["mc.nu.iscc"] & (np.abs(ovrl["mc.nu.pdg"]) == 14)
nue_cc = ovrl["mc.nu.iscc"] & (np.abs(ovrl["mc.nu.pdg"]) == 12)
nu_nc = np.invert(ovrl["mc.nu.iscc"])

toplot = [var[numu_cc], var[nue_cc], var[nu_nc]]
weights = [np.repeat(ovrl_pot_scale, np.sum(numu_cc)), 
           np.repeat(ovrl_pot_scale, np.sum(nue_cc)), 
           np.repeat(ovrl_pot_scale, np.sum(nu_nc))]

_ = plt.hist(toplot,
            weights=weights,
            label=[r'$\nu_\mu$ CC', r'$\nu_e$ CC', r'$\nu$ NC'],
            histtype="step",
            bins=np.linspace(0, 6, 19))
plt.legend()
plt.yscale("log")
plt.ylabel("True $\\nu$`s / 6.6e20 POT")
plt.xlabel("$\\nu$ Energy [GeV]")

if do_save: plt.savefig(savedir + "all_nu.png")

In [ ]:
# Show all the numu CC neutrinos + cosmics at truth level

var = ovrl["mc.nu.E"]*0 # copy
var[numu_cc] = np.sqrt(ovrl["mc.nu.prim.genp.x"]**2 + ovrl["mc.nu.prim.genp.y"]**2 + ovrl["mc.nu.prim.genp.z"]**2)[np.abs(ovrl["mc.nu.prim.pdg"])==13].flatten()

toplot = [var[numu_cc]]
weights = [np.repeat(ovrl_pot_scale, np.sum(numu_cc))]

# now for the cosmics
var_ovrl_cosmic = np.sqrt(ovrl["true_particles.genp.x"]**2 + ovrl["true_particles.genp.y"]**2 + ovrl["true_particles.genp.z"]**2)
ovrl_is_cosmic = (ovrl["true_particles.interaction_id"] < 0) & (np.abs(ovrl["true_particles.pdg"]) == 13) & (ovrl["true_particles.start.x"] > -9998)

var_intc_cosmic = np.sqrt(intc["true_particles.genp.x"]**2 + intc["true_particles.genp.y"]**2 + intc["true_particles.genp.z"]**2)
intc_is_cosmic = (intc["true_particles.interaction_id"] < 0) & (np.abs(intc["true_particles.pdg"]) == 13) & (intc["true_particles.start.x"] > -9998)

toplot += [np.hstack([var_ovrl_cosmic[ovrl_is_cosmic].flatten(), var_intc_cosmic[intc_is_cosmic].flatten()])]
weights += [np.hstack([np.repeat(ovrl_pot_scale, np.sum(ovrl_is_cosmic.flatten())), np.repeat(intc_pot_scale, np.sum(intc_is_cosmic.flatten()))])]

_ = plt.hist(toplot,
            weights=weights,
            label=[r'$\nu_\mu$ CC $\mu$', r'Cosmic $\mu$'],
            histtype="step",
            bins=np.linspace(0, 6, 19))
plt.legend(loc="lower left")
plt.yscale("log")
plt.ylabel("True $\\mu$`s / 6.6e20 POT")
plt.xlabel("$\\mu$ Momentum [GeV]")

print("%.2e" % (ovrl_pot_scale * np.sum(numu_cc)))
print("%.2e" % (ovrl_pot_scale * np.sum(ovrl_is_cosmic.flatten())))

print("%.2e" % (intc_pot_scale * np.sum(intc_is_cosmic.flatten())))

if do_save: plt.savefig(savedir + "all_mu.png")

In [ ]:
# Show all the numu CC neutrinos + cosmics at truth level

visible_pdg = (np.abs(ovrl["mc.nu.prim.pdg"]) == 13)|\
    (np.abs(ovrl["mc.nu.prim.pdg"]) == 11)|\
    (np.abs(ovrl["mc.nu.prim.pdg"]) == 22)|\
    (np.abs(ovrl["mc.nu.prim.pdg"]) == 2212)|\
    (np.abs(ovrl["mc.nu.prim.pdg"]) == 211)|\
    (np.abs(ovrl["mc.nu.prim.pdg"]) == 111)

var = np.sqrt(ovrl["mc.nu.prim.genp.x"]**2 + ovrl["mc.nu.prim.genp.y"]**2 + ovrl["mc.nu.prim.genp.z"]**2)[visible_pdg].max()

toplot = [var[numu_cc], var[nue_cc], var[nu_nc]]
weights = [np.repeat(ovrl_pot_scale, np.sum(numu_cc)), 
           np.repeat(ovrl_pot_scale, np.sum(nue_cc)), 
           np.repeat(ovrl_pot_scale, np.sum(nu_nc))]

# now for the cosmics
var_ovrl_cosmic = np.sqrt(ovrl["true_particles.genp.x"]**2 + ovrl["true_particles.genp.y"]**2 + ovrl["true_particles.genp.z"]**2)
ovrl_is_cosmic = (ovrl["true_particles.interaction_id"] < 0) & (np.abs(ovrl["true_particles.pdg"]) == 13) & (ovrl["true_particles.start.x"] > -9998)

var_intc_cosmic = np.sqrt(intc["true_particles.genp.x"]**2 + intc["true_particles.genp.y"]**2 + intc["true_particles.genp.z"]**2)
intc_is_cosmic = (intc["true_particles.interaction_id"] < 0) & (np.abs(intc["true_particles.pdg"]) == 13) & (intc["true_particles.start.x"] > -9998)

toplot += [np.hstack([var_ovrl_cosmic[ovrl_is_cosmic].flatten(), var_intc_cosmic[intc_is_cosmic].flatten()])]
weights += [np.hstack([np.repeat(ovrl_pot_scale, np.sum(ovrl_is_cosmic.flatten())), np.repeat(intc_pot_scale, np.sum(intc_is_cosmic.flatten()))])]

_ = plt.hist(toplot,
            weights=weights,
            label=[r'$\nu_\mu$ CC', r'$\nu_e$ CC', r'$\nu$ NC', r'Cosmic'],
            histtype="step",
            bins=np.linspace(0, 6, 19))
plt.legend(loc="lower left", ncol=2, columnspacing=0.4)
plt.yscale("log")
plt.ylabel("Events / 6.6e20 POT")
plt.xlabel("Largest Visible Particle Momentum [GeV]")

ylim = plt.ylim()

if do_save: plt.savefig(savedir + "all_nu_cosmic.png")

In [ ]:
# Show all the nus
toplot = [var[numu_cc], var[nue_cc], var[nu_nc]]
weights = [np.repeat(ovrl_pot_scale, np.sum(numu_cc)), 
           np.repeat(ovrl_pot_scale, np.sum(nue_cc)), 
           np.repeat(ovrl_pot_scale, np.sum(nu_nc))]

_ = plt.hist(toplot,
            weights=weights,
            label=[r'$\nu_\mu$ CC', r'$\nu_e$ CC', r'$\nu$ NC'],
            histtype="step",
            bins=np.linspace(0, 6, 19))
plt.legend(loc="lower left", ncol=2, columnspacing=0.4)
plt.yscale("log")
plt.ylabel("Events / 6.6e20 POT")
plt.xlabel("Largest Visible Particle Momentum [GeV]")
plt.ylim(ylim)

if do_save: plt.savefig(savedir + "largestp_all_nu.png")

In [ ]:
toplot = np.hstack([var_ovrl_cosmic[ovrl_is_cosmic].flatten(), var_intc_cosmic[intc_is_cosmic].flatten()])
weights = np.hstack([np.repeat(ovrl_pot_scale, np.sum(ovrl_is_cosmic.flatten())), np.repeat(intc_pot_scale, np.sum(intc_is_cosmic.flatten()))])
intime = np.hstack([selection.InBeamTrue(ovrl["true_particles.startT"])[ovrl_is_cosmic].flatten(),
                   selection.InBeamTrue(intc["true_particles.startT"])[intc_is_cosmic].flatten()])

cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
_ = plt.hist([toplot[np.invert(intime)], toplot[intime]],
            weights=[weights[np.invert(intime)]/4., weights[intime]],
            label=[r'Out-of-Time Cosmic', r'In-Time Cosmic'],
            histtype="step",
            color=[cycle[3], cycle[4]],
            bins=np.linspace(0, 6, 19))
plt.legend(loc="lower left", ncol=2, columnspacing=0.4)
plt.yscale("log")
plt.ylabel("Cosmics / 6.6e20 POT")
plt.xlabel("Largest Visible Particle Momentum [GeV]")

plt.ylim(ylim)

if do_save: plt.savefig(savedir + "largestp_all_cosmic.png")

In [ ]:
toplot = [ovrl["crt_hits.pe"][selection.InBeam(ovrl["crt_hits.time"])].flatten(), 
          intc["crt_hits.pe"][selection.InBeam(intc["crt_hits.time"])].flatten()]
weights = [np.repeat(ovrl_pot_scale, len(toplot[0])), np.repeat(intc_pot_scale, len(toplot[1]))]

_ = plt.hist(toplot,
             weights=weights,
             label=["$\\nu$+Cosmic Spills", "Intime Cosmic Spills"],
            histtype="step")
plt.xlabel("PE Of In-Beam-Time CRT Hits")
plt.legend()
plt.ylabel("Events / 6.6e20 POT")

if do_save: plt.savefig(savedir + "intime_hit_pe.png")

In [ ]:
var = ovrl["slc.truth.E"]
when = is_fid_true & is_numu_cc & ovrl["slc.truth.match_is_primary"]

n,_,_ = plt.hist([var[when], var[when & np.invert(ovrl["slc.is_clear_cosmic"])]],
            label=["All Pandora Slices", "Not Clear Cosmic"],
             weights=[np.repeat(ovrl_pot_scale, np.sum(when)), 
                      np.repeat(ovrl_pot_scale, np.sum(when & np.invert(ovrl["slc.is_clear_cosmic"])))],
             bins=np.linspace(0, 3, 21),
            histtype="step")
plt.legend()
plt.xlabel("$\\nu$ Energy [GeV]")
plt.ylabel("Slices / 6.6e20 POT")
    
plt.gca().ticklabel_format(axis="y", style="sci", scilimits=(0,0))


rej = 1 - np.sum(n[-1]) / np.sum(n[0])

plt.text(0.75, 0.5, "Fiducial CC $\\nu_\\mu$`s\n %.1f%% Rejected" % (rej*100), horizontalalignment='center',fontsize=14, transform=plt.gca().transAxes)


if do_save: plt.savefig(savedir + "Pandora_Clear_Cosmic_Neutrino.png")

In [ ]:
var_ovrl = ovrl["slc.reco.trk.truth.p.length"][np.abs(ovrl["slc.reco.trk.truth.p.pdg"]) == 13].max()
when_ovrl = is_cosmic & (np.abs(ovrl["slc.reco.trk.truth.p.pdg"]) == 13).any()
var_intc = intc["slc.reco.trk.truth.p.length"][np.abs(intc["slc.reco.trk.truth.p.pdg"]) == 13].max()
when_intc = (np.abs(intc["slc.reco.trk.truth.p.pdg"]) == 13).any()

toplot = [np.hstack([var_ovrl[when_ovrl], var_intc[when_intc]]), 
              np.hstack([var[when_ovrl & np.invert(ovrl["slc.is_clear_cosmic"])], 
                         var_intc[when_intc & np.invert(intc["slc.is_clear_cosmic"])]])]

weights = [np.hstack([np.repeat(ovrl_pot_scale, np.sum(when_ovrl)), np.repeat(intc_pot_scale, np.sum(when_intc))]), 
            np.hstack([np.repeat(ovrl_pot_scale, np.sum(when_ovrl & np.invert(ovrl["slc.is_clear_cosmic"]))),
                    np.repeat(intc_pot_scale, np.sum(when_intc & np.invert(intc["slc.is_clear_cosmic"])))])]

n,_,_ = plt.hist(toplot,
            label=["All Pandora Slices", "Not Clear Cosmic"],
             weights=weights,
             bins=np.linspace(0, 600, 21),
            histtype="step")

rej = 1 - np.sum(n[-1]) / np.sum(n[0])

plt.legend(loc="lower left")
plt.xlabel("Cosmic $\\mu$ Length [cm]")
plt.ylabel("Slices / 6.6e20 POT")
plt.yscale("log")

plt.text(0.25, 0.6, "Sliced Cosmic $\\mu$`s\n %.1f%% Rejected" % (rej*100), horizontalalignment='center',fontsize=14, transform=plt.gca().transAxes)

if do_save: plt.savefig(savedir + "Pandora_Clear_Cosmic_Cosmic.png")

In [ ]:
var_ovrl = np.sqrt(
    (ovrl["slc.vertex.x"] - ovrl["slc.truth.position.x"])**2 +\
    (ovrl["slc.vertex.y"] - ovrl["slc.truth.position.y"])**2 +\
    (ovrl["slc.vertex.z"] - ovrl["slc.truth.position.z"])**2
    )

when = is_numu_cc & ovrl["slc.truth.match_is_primary"]

_ =plt.hist(var_ovrl[when],
        weights=np.repeat(ovrl_pot_scale, np.sum(when)),
        bins=np.linspace(0, 100, 101),
            label=r'$\nu_\mu$ CC Slices', 
        )
plt.legend()
plt.xlabel("Distance from True to Reconstructed Vertex [cm]")
plt.ylabel("Slices / 6.6e20 POT")
plt.yscale("log")

if do_save: plt.savefig(savedir + "numucc_vtx_dist.png")

In [ ]:
var_ovrl = ovrl["slc.vertex.y"] # np.maximum(ovrl["slc.reco.trk.end.y"], ovrl["slc.reco.trk.start.y"])
when_ovrl = np.invert(ovrl["slc.is_clear_cosmic"]) & ovrl["slc.truth.match_is_primary"] #& (np.abs(ovrl["slc.reco.trk.truth.p.pdg"]) == 13)

var_intc = intc["slc.vertex.y"]
when_intc = np.invert(intc["slc.is_clear_cosmic"]) & intc["slc.truth.match_is_primary"] #& (np.abs(intc["slc.reco.trk.truth.p.pdg"]) == 13)

_ =plt.hist([var_ovrl[when_ovrl & is_numu_cc].flatten(),
             np.hstack([var_ovrl[when_ovrl & is_cosmic].flatten(), var_intc[when_intc].flatten()])],
            weights=[np.repeat(ovrl_pot_scale, np.sum((when_ovrl & is_numu_cc).flatten())),
                    np.hstack([np.repeat(ovrl_pot_scale, np.sum((when_ovrl & is_cosmic).flatten())), 
                              np.repeat(intc_pot_scale, np.sum(when_intc.flatten()))])],
        bins=np.linspace(150, 200, 26),
            label=[r'$\nu_\mu$ CC $\mu$', r'Cosmic $\mu$'], 
            histtype="step",
        )
plt.legend(loc="upper left")
plt.xlabel("Vertex Y Coordinate [cm]")
plt.ylabel("Slices / 6.6e20 POT")
plt.yscale("log")
        
plt.text(0.3, 0.5, "Post Clear-Cosmic Cut", fontsize=14, horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes,)
if do_save: plt.savefig(savedir + "numucc_vertex_maxy_cosmic.png")

In [ ]:
var_ovrl = np.maximum(ovrl["slc.reco.trk.end.y"], ovrl["slc.reco.trk.start.y"])

when = is_numu_cc & ovrl["slc.truth.match_is_primary"] & (np.abs(ovrl["slc.reco.trk.truth.p.pdg"]) == 13)

_ =plt.hist([var_ovrl[when & ovrl["slc.reco.trk.truth.p.contained"]].flatten(), 
            var_ovrl[when & (ovrl["slc.reco.trk.truth.p.wallout"] == 1)].flatten()],
        bins=np.linspace(180, 200, 21),
            label=[r'$\nu_\mu$ CC $\mu$`s (Contained)', r'$\nu_\mu$ CC $\mu$`s (Exiting out of Top)'], 
            histtype="step",
        )
plt.legend(loc="upper left")
plt.xlabel("Maximum Y Coordinate [cm]")
plt.ylabel("Tracks (Not Normalized)")
plt.yscale("log")

if do_save: plt.savefig(savedir + "numucc_mutrk_maxy.png")

In [ ]:
var_ovrl = np.minimum(ovrl["slc.reco.trk.end.y"], ovrl["slc.reco.trk.start.y"])

when = is_numu_cc & ovrl["slc.truth.match_is_primary"] & (np.abs(ovrl["slc.reco.trk.truth.p.pdg"]) == 13)

_ =plt.hist([var_ovrl[when & ovrl["slc.reco.trk.truth.p.contained"]].flatten(), 
            var_ovrl[when & (ovrl["slc.reco.trk.truth.p.wallout"] == 2)].flatten()],
        bins=np.linspace(-200, -180, 21),
            label=[r'$\nu_\mu$ CC $\mu$`s (Contained)', r'$\nu_\mu$ CC $\mu$`s (Exiting out of Bottom)'], 
            histtype="step",
        )
plt.legend(loc="upper right")
plt.xlabel("Minimum Y Coordinate [cm]")
plt.ylabel("Tracks (Not Normalized)")
plt.yscale("log")

if do_save: plt.savefig(savedir + "numucc_mutrk_miny.png")

In [ ]:
var_ovrl = np.maximum(ovrl["slc.reco.trk.end.x"], ovrl["slc.reco.trk.start.x"])

when = is_numu_cc & ovrl["slc.truth.match_is_primary"] & (np.abs(ovrl["slc.reco.trk.truth.p.pdg"]) == 13)

_ =plt.hist([var_ovrl[when & ovrl["slc.reco.trk.truth.p.contained"]].flatten(), 
            var_ovrl[when & (ovrl["slc.reco.trk.truth.p.wallout"] == 4)].flatten()],
        bins=np.linspace(180, 200, 21) - 0.85,
            label=[r'$\nu_\mu$ CC $\mu$`s (Contained)', r'$\nu_\mu$ CC $\mu$`s (Exiting out of Right Wall)'], 
            histtype="step",
        )
plt.legend(loc="upper left")
plt.xlabel("Maximum X Coordinate [cm]")
plt.ylabel("Tracks (Not Normalized)")
plt.yscale("log")

if do_save: plt.savefig(savedir + "numucc_mutrk_maxx.png")

In [ ]:
var_ovrl = np.minimum(ovrl["slc.reco.trk.end.x"], ovrl["slc.reco.trk.start.x"])

when = is_numu_cc & ovrl["slc.truth.match_is_primary"] & (np.abs(ovrl["slc.reco.trk.truth.p.pdg"]) == 13)

_ =plt.hist([var_ovrl[when & ovrl["slc.reco.trk.truth.p.contained"]].flatten(), 
            var_ovrl[when & (ovrl["slc.reco.trk.truth.p.wallout"] == 3)].flatten()],
        bins=np.linspace(-200, -180, 21) + 0.85,
            label=[r'$\nu_\mu$ CC $\mu$`s (Contained)', r'$\nu_\mu$ CC $\mu$`s (Exiting out of Left Wall)'], 
            histtype="step",
        )
plt.legend(loc="upper right")
plt.xlabel("Minimum X Coordinate [cm]")
plt.ylabel("Tracks (Not Normalized)")
plt.yscale("log")

if do_save: plt.savefig(savedir + "numucc_mutrk_minx.png")

In [ ]:
var_ovrl = np.minimum(ovrl["slc.reco.trk.end.z"], ovrl["slc.reco.trk.start.z"])

when = is_numu_cc & ovrl["slc.truth.match_is_primary"] & (np.abs(ovrl["slc.reco.trk.truth.p.pdg"]) == 13)

_ =plt.hist([var_ovrl[when & ovrl["slc.reco.trk.truth.p.contained"]].flatten(), 
            var_ovrl[when & (ovrl["slc.reco.trk.truth.p.wallout"] == 5)].flatten()],
        bins=np.linspace(0, 20, 21),
            label=[r'$\nu_\mu$ CC $\mu$`s (Contained)', r'$\nu_\mu$ CC $\mu$`s (Exiting out of Front)'], 
            histtype="step",
        )
plt.legend(loc="upper right")
plt.xlabel("Minimum Z Coordinate [cm]")
plt.ylabel("Tracks (Not Normalized)")
plt.yscale("log")

if do_save: plt.savefig(savedir + "numucc_mutrk_minz.png")

In [ ]:
var_ovrl = np.maximum(ovrl["slc.reco.trk.end.z"], ovrl["slc.reco.trk.start.z"])

when = is_numu_cc & ovrl["slc.truth.match_is_primary"] & (np.abs(ovrl["slc.reco.trk.truth.p.pdg"]) == 13)

_ =plt.hist([var_ovrl[when & ovrl["slc.reco.trk.truth.p.contained"]].flatten(), 
            var_ovrl[when & (ovrl["slc.reco.trk.truth.p.wallout"] == 6)].flatten()],
        bins=np.linspace(480, 500, 21),
            label=[r'$\nu_\mu$ CC $\mu$`s (Contained)', r'$\nu_\mu$ CC $\mu$`s (Exiting out of Back)'], 
            histtype="step",
        )
plt.legend(loc="upper left")
plt.xlabel("Maximum Z Coordinate [cm]")
plt.ylabel("Tracks (Not Normalized)")
plt.yscale("log")

if do_save: plt.savefig(savedir + "numucc_mutrk_maxz.png")

In [ ]:
class Category:
    def __init__(self, w, l):
        self.when = w
        self.label = l
    def when(self, d):
        return self.when(d)
    
CATEGORIES = {
    "numucc": Category(lambda d: d["slc.numu_cc"] & d["slc.truth.match_is_primary"], "$\\nu_\\mu$ CC"),
    "numu_other": Category(lambda d: np.invert(d["slc.cosmic"]) & d["slc.truth.match_is_primary"] & np.invert( d["slc.numu_cc"]),
                          "Other $\\nu$"),
    "cosmic": Category(lambda d: d["slc.cosmic"], "Cosmic $\\mu$"),
    "intime_cosmic": Category(lambda d: d["slc.cosmic"] & selection.InBeamTrue(d["slc.ptrk.truth.p.startT"]),
                             "Cosmic in Beam Time"),
    "intime_cosmic_loose": Category(lambda d: d["slc.cosmic"] & (d["slc.ptrk.truth.p.startT"] > -0.2) & (d["slc.ptrk.truth.p.startT"] < 2.),
                             "Cosmic in Beam Time"),
    "outtime_cosmic":  Category(lambda d: d["slc.cosmic"] & np.invert(selection.InBeamTrue(d["slc.ptrk.truth.p.startT"])),
                             "Cosmic out of Beam Time")
}

def plot(fig, var, cut, bins, cutval, cut_is_gt, xlabel, precut=None, plotcut=None, text=None, textX=0.5, textY=0.5, 
         categories=["numucc", "cosmic"]):
    var_ovrl = var(ovrl)
    var_intc = var(intc)
    cut_ovrl = cut(ovrl)
    cut_intc = cut(intc)
    
    if not precut:
        precut_ovrl = np.repeat(True, len(var_ovrl))
        precut_intc = np.repeat(True, len(var_intc))
    else:
        precut_ovrl = precut(ovrl)
        precut_intc = precut(intc)
        
    if not plotcut:
        plotcut_ovrl = np.repeat(True, len(var_ovrl))
        plotcut_intc = np.repeat(True, len(var_intc))
    else:
        plotcut_ovrl = plotcut(ovrl)
        plotcut_intc = plotcut(intc)
    
    categories = [CATEGORIES[c] for c in categories]
    toplot = [np.hstack([var_ovrl[c.when(ovrl) & precut_ovrl & plotcut_ovrl], var_intc[c.when(intc) & precut_intc & plotcut_intc]]) for c in categories]
    weights = [np.hstack([np.repeat(ovrl_pot_scale, np.sum(c.when(ovrl) & precut_ovrl & plotcut_ovrl)), np.repeat(intc_pot_scale, np.sum(c.when(intc) & precut_intc & plotcut_intc))]) for c in categories]

    n,_,p = fig.hist(toplot,
                 weights=weights,
                label=[c.label for c in categories],
                 bins=bins,
                histtype="step")
    fig.legend()
    fig.ylabel("Slices / 6.6e20 POT")
    _ = fig.xlabel(xlabel)
    fig.gca().ticklabel_format(axis="y", style="sci", scilimits=(0,0))


    ncosmic = n[1]
    nnu = n[0]
    cosmic_color = p[1][0].get_edgecolor()
    nu_color = p[0][0].get_edgecolor()
    hmax = np.max(np.hstack(n))

    ncosmic = intc_pot_scale * np.sum(CATEGORIES["cosmic"].when(intc) & precut_intc) +\
              ovrl_pot_scale * np.sum(CATEGORIES["cosmic"].when(ovrl) & precut_ovrl)
    ncosmicrej = intc_pot_scale * np.sum(CATEGORIES["cosmic"].when(intc) & precut_intc & np.invert(cut_intc)) +\
                 ovrl_pot_scale * np.sum(CATEGORIES["cosmic"].when(ovrl) & precut_ovrl & np.invert(cut_ovrl))

    rej = ncosmicrej / ncosmic

    nnu     = ovrl_pot_scale * np.sum(CATEGORIES["numucc"].when(ovrl) & precut_ovrl)
    nnupass = ovrl_pot_scale * np.sum(CATEGORIES["numucc"].when(ovrl) & precut_ovrl & cut_ovrl)
    eff = nnupass / nnu

    cutval_a = (cutval - plt.xlim()[0]) / (plt.xlim()[1] - plt.xlim()[0])
    
    l = mpl.lines.Line2D((cutval_a,cutval_a), (0., 1.1), color="black", transform=fig.gca().transAxes,)
    l.set_clip_on(False)
    fig.gca().add_line(l)
    
    arrow_lo = max(0., cutval_a - 0.3)
    arrow_hi = min(1., cutval_a + 0.3)
    text_lo = (cutval_a + arrow_lo) / 2.
    text_hi = max((cutval_a + arrow_hi) / 2., text_lo + 0.3)
    
    if cut_is_gt:
        bkg_arrow = arrow_lo
        bkg_text = text_lo
        sig_arrow = arrow_hi
        sig_text = text_hi
    else:
        bkg_arrow = arrow_hi
        bkg_text = text_hi
        sig_arrow = arrow_lo
        sig_text = text_lo
    
    fig.gca().annotate("", xytext=(bkg_arrow, 1.05), xy=(cutval_a, 1.05),
                 arrowprops=dict(arrowstyle="<-", color=cosmic_color), xycoords=fig.gca().transAxes,)
    fig.text(bkg_text, 1.1, "Cos. Rej. %.1f%%" % (rej*100),
              transform=fig.gca().transAxes, fontsize=12, color=cosmic_color, horizontalalignment='center')
    
    fig.gca().annotate("", xytext=(sig_arrow, 1.05), xy=(cutval_a, 1.05),
                 arrowprops=dict(arrowstyle="<-", color=nu_color), xycoords=fig.gca().transAxes,)
    fig.text(sig_text, 1.1, "Sel. Eff. %.1f%%" % (eff*100), 
             transform=fig.gca().transAxes, fontsize=12, color=nu_color, horizontalalignment='center')
    
    if text:
        fig.text(textX, textY, text, fontsize=14, horizontalalignment='center', verticalalignment='center', transform=fig.gca().transAxes,)

In [ ]:
f = plt.figure(0)
var = lambda d: d["slc.nu_score"]
cut = selection.nu_score
bins = np.linspace(0, 1, 26)
xlabel = "Pandora $\\nu$-Score"
precut = lambda d: np.invert(d["slc.is_clear_cosmic"]) & selection.fid(d)
text="Fiducial\nNon-Clear Cosmic"

plot(plt, var, cut, bins, 0.4, True, xlabel, precut=precut, text=text, textX=0.2, textY=0.9)

if do_save: plt.savefig(savedir + "cutvars/nu_score.png")

In [ ]:
# make a profile plot
varx = ovrl["slc.truth.E"]*0 # make a copy
varx[is_numu_cc] = ovrl["slc.truth.prim.length"][np.abs(ovrl["slc.truth.prim.pdg"]) == 13].flatten()
vary = ovrl["slc.nu_score"]

N,xbin,ybin = np.histogram2d(varx[fid_ovrl], vary[fid_ovrl], bins=[np.linspace(0, 500, 26), np.linspace(0, 1, 21)])

# convert to profile
xvals = (xbin[:-1] + xbin[1:]) / 2.
ycenters = (ybin[:-1] + ybin[1:]) / 2.
yvals = np.sum(N * ycenters, axis=1) / np.sum(N, axis=1)
_ = plt.plot(xvals, yvals)
plt.ylim(0.5, 0.7)
plt.ylabel("Mean Pandora $\\nu$-Score")
plt.xlabel("True $\\nu_\\mu$ CC Muon Length")

if do_save: plt.savefig(savedir + "nu_score_profile.png")

In [ ]:
plt.figure(0)
var = lambda d: d["slc.fmatch.time"]
cut = selection.f_time
xlabel = "Flash Time [$\\mu$s]"
bins = np.linspace(-0.2, 2,23)
precut = lambda d: np.invert(d["slc.is_clear_cosmic"]) & selection.fid(d)
text="Fiducial\nNon-Clear Cosmic"
plot(plt, var, cut, bins, 1.8, False, xlabel, precut=precut, text=text, textX=0.5, textY=0.5,
    categories=["numucc", "intime_cosmic_loose"])
    
# also draw a line on the low part of the cut
cutval = 0.
cutval_a = (cutval - plt.xlim()[0]) / (plt.xlim()[1] - plt.xlim()[0])
l = mpl.lines.Line2D((cutval_a,cutval_a), (0., 1.1), color="black", transform=plt.gca().transAxes,)
l.set_clip_on(False)
plt.gca().add_line(l)

if do_save: plt.savefig(savedir + "cutvars/fmatch_time.png")

In [ ]:
plt.figure(0)
var = lambda d: d["slc.fmatch.score"]
cut = selection.f_score
xlabel = "Flash Score"
bins = np.linspace(0, 25,26)
precut = lambda d: np.invert(d["slc.is_clear_cosmic"]) & selection.fid(d)
text="Fiducial\nNon-Clear Cosmic"
plot(plt, var, cut, bins, 7, False, xlabel, precut=precut, text=text, textX=0.7, textY=0.5,
    categories=["numucc", "outtime_cosmic", "intime_cosmic"])
if do_save: plt.savefig(savedir + "cutvars/fmatch.png")
    
plt.figure(1)
precut = lambda d: selection.nu_score(d) & selection.fid(d) 
text="Fiducial\nNon-Clear Cosmic\nPost $\\nu$-Score"
plot(plt, var, cut, bins, 7, False, xlabel, precut=precut, text=text, textX=0.7, textY=0.5,
     categories=["numucc", "outtime_cosmic", "intime_cosmic"])
if do_save: plt.savefig(savedir + "cutvars/fmatch_post_nu_score.png")

In [ ]:
plt.figure(0)
var = lambda d: d["slc.ptrk.crttrack.angle"]
cut = selection.crttrack
xlabel = "CRT Track Angle [rad]"
bins = np.linspace(0, 1, 21)
plotcut = lambda d: np.invert(np.isnan(d["slc.ptrk.crttrack.angle"]))
precut = lambda d: np.invert(d["slc.is_clear_cosmic"]) & selection.fid(d) & selection.ptrk(d)
text="Fiducial\nNon-Clear Cosmic\nHas Primary Track"
plot(plt, var, cut, bins, 0.05, True, xlabel, precut=precut, plotcut=plotcut, text=text, textX=0.7, textY=0.5)
if do_save: plt.savefig(savedir + "cutvars/crttrack.png")

plt.figure(1)
precut = lambda d: selection.nu_score(d) & selection.fid(d) & selection.ptrk(d)
text="Fiducial\nNon-Clear Cosmic\nHas Primary Track\nNu-Score"
plot(plt, var, cut, bins, 0.05, True, xlabel, precut=precut, plotcut=plotcut, text=text, textX=0.7, textY=0.5)
if do_save: plt.savefig(savedir + "cutvars/crttrack_post_nu_score.png")

plt.figure(2)
precut = lambda d: selection.nu_score(d) & selection.fid(d) & selection.f_score(d) & selection.f_time(d) & selection.ptrk(d)
text="Fiducial\nNon-Clear Cosmic\nHas Primary Track\nNu-Score\nFlash Match"
plot(plt, var, cut, bins, 0.05, True, xlabel, precut=precut, plotcut=plotcut, text=text, textX=0.7, textY=0.5)
if do_save: plt.savefig(savedir + "cutvars/crttrack_post_nu_score_fmatch.png")

#if do_save: plt.savefig(savedir + "crt_track_angle.png")

In [ ]:
plt.figure(0)
var = lambda d: d["slc.ptrk.crthit.distance"]
cut = selection.crthit
xlabel = "CRT Hit Distance (Only Hits Outside Beam Time) [cm]"
bins = np.linspace(0, 100, 21)
plotcut = lambda d: np.invert(np.isnan(d["slc.ptrk.crthit.distance"])) & np.invert(selection.InBeam(d["slc.ptrk.crthit.hit.time"]))
precut = lambda d: np.invert(d["slc.is_clear_cosmic"]) & selection.fid(d) & selection.ptrk(d)
text="Fiducial\nNon-Clear Cosmic\nHas Primary Track"
plot(plt, var, cut, bins, 5, True, xlabel, precut=precut, plotcut=plotcut, text=text, textX=0.7, textY=0.5)
if do_save: plt.savefig(savedir + "cutvars/crthit.png")

plt.figure(1)
precut = lambda d: selection.nu_score(d) & selection.fid(d) & selection.ptrk(d)
text="Fiducial\nNon-Clear Cosmic\nHas Primary Track\nNu-Score"
plot(plt, var, cut, bins, 5, True, xlabel, precut=precut, plotcut=plotcut, text=text, textX=0.7, textY=0.5)
if do_save: plt.savefig(savedir + "cutvars/crthit_post_nu_score.png")

plt.figure(2)
precut = lambda d: selection.nu_score(d) & selection.fid(d) & selection.f_score(d) & selection.f_time(d) & selection.ptrk(d)
text="Fiducial\nNon-Clear Cosmic\nHas Primary Track\nNu-Score\nFlash Match"
plot(plt, var, cut, bins, 5, True, xlabel, precut=precut, plotcut=plotcut, text=text, textX=0.7, textY=0.5)
if do_save: plt.savefig(savedir + "cutvars/crthit_post_nu_score_fmatch.png")

plt.figure(3)
precut = lambda d: selection.nu_score(d) & selection.fid(d) & selection.f_score(d) & selection.f_time(d) & selection.crttrack(d) & selection.ptrk(d)
text="Fiducial\nNon-Clear Cosmic\nHas Primary Track\nNu-Score\nFlash Match\nCRT Track Match" 
plot(plt, var, cut, bins, 5, True, xlabel, precut=precut, plotcut=plotcut, text=text, textX=0.7, textY=0.5)
if do_save: plt.savefig(savedir + "cutvars/crthit_post_nu_score_fmatch_crttrack.png")

#if do_save: plt.savefig(savedir + "crt_track_angle.png")

In [ ]:
var_ovrl = ovrl["slc.ptrk.recop"]
var_intc = intc["slc.ptrk.recop"]

when_ovrl = fid_ovrl & nu_score_ovrl & f_time_ovrl & f_score_ovrl & ptrk_ovrl & crttrack_ovrl & crthit_ovrl
when_intc = fid_intc & nu_score_intc & f_time_intc & f_score_intc & ptrk_intc & crttrack_intc & crthit_intc

categories = [CATEGORIES[c] for c in ["numucc", "cosmic", "numu_other"]]

toplot = [np.hstack([var_ovrl[c.when(ovrl) & when_ovrl], var_intc[c.when(intc) & when_intc]]) for c in categories]
weights = [np.hstack([np.repeat(ovrl_pot_scale, np.sum(c.when(ovrl) & when_ovrl)), np.repeat(intc_pot_scale, np.sum(c.when(intc) & when_intc))]) for c in categories]

_ = plt.hist(toplot,
             weights=weights,
            label=[c.label for c in categories],
            bins=np.linspace(0, 2, 21),
            histtype="step")
_ = plt.legend()
    
plt.gca().ticklabel_format(axis="y", style="sci", scilimits=(0,0))


plt.text(0.7, 0.5, 'TPC Pre-Selection\nFlash Matching\nMuon Track Identification\nCRT Matching', horizontalalignment='center', verticalalignment='center',fontsize=14, transform=plt.gca().transAxes)
plt.xlabel("Reconstructed Muon Momentum [GeV]")
plt.ylabel("Slices / 6.6e20")
if do_save: plt.savefig(savedir + "overlay_spectra.png")

In [ ]:
var_ovrl = ovrl["slc.ptrk.recop"]
var_intc = intc["slc.ptrk.recop"]

when_ovrl = fid_ovrl & nu_score_ovrl & f_time_ovrl & f_score_ovrl & ptrk_ovrl & crttrack_ovrl & crthit_ovrl & crtveto_ovrl
when_intc = fid_intc & nu_score_intc & f_time_intc & f_score_intc & ptrk_intc & crttrack_intc & crthit_intc & crtveto_intc

categories = [CATEGORIES[c] for c in ["numucc", "cosmic", "numu_other"]]

toplot = [np.hstack([var_ovrl[c.when(ovrl) & when_ovrl], var_intc[c.when(intc) & when_intc]]) for c in categories]
weights = [np.hstack([np.repeat(ovrl_pot_scale, np.sum(c.when(ovrl) & when_ovrl)), np.repeat(intc_pot_scale, np.sum(c.when(intc) & when_intc))]) for c in categories]

_ = plt.hist(toplot,
             weights=weights,
            label=[c.label for c in categories],
            bins=np.linspace(0, 2, 21),
            histtype="step")
_ = plt.legend()
    
plt.gca().ticklabel_format(axis="y", style="sci", scilimits=(0,0))


plt.text(0.7, 0.5, 'TPC Pre-Selection\nFlash Matching\nMuon Track Identification\nCRT Veto + Matching', horizontalalignment='center', verticalalignment='center',fontsize=14, transform=plt.gca().transAxes)
plt.xlabel("Reconstructed Muon Momentum [GeV]")
plt.ylabel("Slices / 6.6e20")
if do_save: plt.savefig(savedir + "overlay_spectra_withveto.png")

In [ ]:
cuts = [
    lambda d: d["slc.is_clear_cosmic"] | True,
    lambda d: selection.fid(d) & selection.nu_score(d),
    lambda d: selection.fid(d) & selection.nu_score(d) & selection.f_score(d) & selection.f_time(d),
    lambda d: selection.fid(d) & selection.nu_score(d) & selection.f_score(d) & selection.f_time(d) & selection.ptrk(d),
    lambda d: selection.fid(d) & selection.nu_score(d) & selection.f_score(d) & selection.f_time(d) & selection.ptrk(d) & selection.crttrack(d) & selection.crthit(d)    
]

def Ntotal(cut, category):
    return ovrl_pot_scale * np.sum(cut(ovrl) & category.when(ovrl)) + intc_pot_scale * np.sum(cut(intc) & category.when(intc))

n_numucc = np.array([Ntotal(cut, CATEGORIES["numucc"]) for cut in cuts])
n_numu_other = np.array([Ntotal(cut, CATEGORIES["numu_other"]) for cut in cuts])
n_cosmic = np.array([Ntotal(cut, CATEGORIES["cosmic"]) for cut in cuts])
cutnames = ["All Slices", "TPC Pre-Selection", "Flash Matching", "Muon Track Identification", "CRT Matching"]

ind = np.arange(len(n_numucc))
width = 0.25

fig, ax = plt.subplots()
ax.barh(ind + 2*width, n_numucc, width, label=CATEGORIES["numucc"].label)
ax.barh(ind + width, n_cosmic, width, label=CATEGORIES["cosmic"].label)
ax.barh(ind + 0*width, n_numu_other, width, label=CATEGORIES["numu_other"].label)

ax.set(yticks=ind + width, yticklabels=cutnames, ylim=[2*width-1, len(cuts)])
ax.legend(loc="lower right")
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xscale("log")
ax.set_xlabel("Neutrino Candidates / 6.6e20 POT")
if do_save: plt.savefig(savedir + "cut_progression.png", bbox_inches='tight')

In [ ]:
when_ovrl = fid_ovrl & nu_score_ovrl & f_time_ovrl & f_score_ovrl & ptrk_ovrl & crttrack_ovrl & crthit_ovrl & crtveto_ovrl
when_intc = fid_intc & nu_score_intc & f_time_intc & f_score_intc & ptrk_intc & crttrack_intc & crthit_intc & crtveto_intc

n_numucc = np.sum(when_ovrl & CATEGORIES["numucc"].when(ovrl)) * ovrl_pot_scale
print("N numu CC: %.2e" % n_numucc)

n_nuother = np.sum(when_ovrl & CATEGORIES["numu_other"].when(ovrl)) * ovrl_pot_scale
print("N nu-other CC: %.2e" % n_nuother)

n_ovrl_cosmic = np.sum(when_ovrl & CATEGORIES["cosmic"].when(ovrl)) * ovrl_pot_scale
print("N ovrl cosmic: %.2e" % n_ovrl_cosmic)

n_intc_cosmic = np.sum(when_intc & CATEGORIES["cosmic"].when(intc)) * intc_pot_scale
print("N intc cosmic: %.2e" % n_intc_cosmic)


In [ ]:
match_index_nu = broadcast_ak(group(ovrl["slc.truth.index"], ovrl["nslc"]), ovrl["mc.nnu"].astype(np.int64))
nu_is_fid = selection.InFV(ovrl["mc.nu.position.x"], ovrl["mc.nu.position.y"], ovrl["mc.nu.position.z"])

nu_is_numu_cc = ovrl["mc.nu.iscc"] & (np.abs((ovrl["mc.nu.pdg"])) == 14)
muon_length = ovrl["mc.nu.E"] * 0 # copy
muon_length[nu_is_numu_cc] = ovrl["mc.nu.prim.length"][np.abs(ovrl["mc.nu.prim.pdg"]) == 13].flatten()
muon_contained = ovrl["mc.nu.E"] * 0 # copy
muon_contained[nu_is_numu_cc] = ovrl["mc.nu.prim.contained"][np.abs(ovrl["mc.nu.prim.pdg"]) == 13].flatten()
muon_contained = muon_contained == 1 # make into bool

when_nu = nu_is_fid & nu_is_numu_cc &\
            ((muon_contained & (muon_length > 50)) |\
             (np.invert(muon_contained) & (muon_length > 100)))

def geteff(when_ovrl):
    # group by neutrino events
    when_ovrl_nu = broadcast_ak(group(when_ovrl & ovrl["slc.truth.match_is_primary"], ovrl["nslc"]), ovrl["mc.nnu"].astype(np.int64))
    nu_reco = (when_ovrl_nu & (match_index_nu == ovrl["mc.nu.index"])).any()
    return np.sum(nu_reco & when_nu) / np.sum(when_nu)
    
def print_row(name, when_ovrl, when_intc):
    eff = geteff(when_ovrl)
    cosmic_bkg = (np.sum(when_ovrl & is_cosmic) * ovrl_pot_scale + np.sum(when_intc) * intc_pot_scale) /\
        (np.sum(when_ovrl & is_numu_cc) * ovrl_pot_scale)
    nu_bkg = np.sum(when_ovrl & is_nu & np.invert(is_numu_cc)) / np.sum(when_ovrl)
    print("%s & %.1f & %.1f & %.1f \\\\" % (name, eff*100, cosmic_bkg*100, nu_bkg*100))

In [ ]:
print_row("No Cut Removed", 
          fid_ovrl & nu_score_ovrl & f_time_ovrl & f_score_ovrl & ptrk_ovrl & crttrack_ovrl & crthit_ovrl,
          fid_intc & nu_score_intc & f_time_intc & f_score_intc & ptrk_intc & crttrack_intc & crthit_intc
         )
print_row("Pandora $\\nu$ Score", 
          fid_ovrl & f_time_ovrl & f_score_ovrl & ptrk_ovrl & crttrack_ovrl & crthit_ovrl,
          fid_intc & f_time_intc & f_score_intc & ptrk_intc & crttrack_intc & crthit_intc
         )
print_row("Flash Match Score", 
          fid_ovrl & nu_score_ovrl & f_time_ovrl & ptrk_ovrl & crttrack_ovrl & crthit_ovrl,
          fid_intc & nu_score_intc & f_time_intc & ptrk_intc & crttrack_intc & crthit_intc
         )
print_row("CRT Track Angle", 
          fid_ovrl & nu_score_ovrl & f_time_ovrl & f_score_ovrl & ptrk_ovrl & crthit_ovrl,
          fid_intc & nu_score_intc & f_time_intc & f_score_intc & ptrk_intc & crthit_intc
         )
print_row("CRT Hit Distance", 
          fid_ovrl & nu_score_ovrl & f_time_ovrl & f_score_ovrl & ptrk_ovrl & crttrack_ovrl,
          fid_intc & nu_score_intc & f_time_intc & f_score_intc & ptrk_intc & crttrack_intc
         )

In [ ]:
print_row("TPC Pre-Selection", 
          fid_ovrl & nu_score_ovrl,
          fid_intc & nu_score_intc 
         )
print_row("PMT Flash Matching",
          fid_ovrl & nu_score_ovrl & f_time_ovrl & f_score_ovrl,
          fid_intc & nu_score_intc & f_time_intc & f_score_intc 
         )
print_row("Muon Track Identification",
          fid_ovrl & nu_score_ovrl & f_time_ovrl & f_score_ovrl & ptrk_ovrl,
          fid_intc & nu_score_intc & f_time_intc & f_score_intc & ptrk_intc
         )
print_row("CRT Matching", 
          fid_ovrl & nu_score_ovrl & f_time_ovrl & f_score_ovrl & ptrk_ovrl & crttrack_ovrl & crthit_ovrl,
          fid_intc & nu_score_intc & f_time_intc & f_score_intc & ptrk_intc & crttrack_intc & crthit_intc
         )